In [23]:
import pandas as pd
import datetime as datetime
import os
# ! pip install xlsxwriter --user

In [36]:
data_path=r'D:\PythonDev\MyQuantFinProject\Data'
data_file='Tech-World-EEM-GLD-10YBond.csv'
data_file='KeyRiskAsset.csv'
isPy=False 
isPriceExcel=False

# Specify Period and Timefame to caculate fund performance and

In [48]:
# If you want to see July-Oct, you have to have last trading day of June
from_str='2023-07-30'  
to_str='2023-10-31' 
timeframe='M'
# timeframe='W-FRI'

# Input data

In [49]:
if isPy:
    print(f"Data csv file in Path: {data_path}")
    data_file=input("CSV file name : ")

    from_str=input("From_date(2020-10-01) : ") 
    to_str=input("To_date(2022-12-31): ") 
    timeframe=input("Timeframe(W-FRI,M,Q,Y): ") 
  
    print(f"Process data from {from_str} to {to_str} in {timeframe} timeframe")
    press_y=input(f"Press y=True and n=False to run analaystics : ") 
    if press_y.lower()=='y':
     ok=True
    else:
     exit()


In [50]:
report_prefix_name=data_file.split('.')[0]
data_file=os.path.join(data_path,data_file)
if os.path.exists(data_file)==False:
    raise Exception(f"not found {data_file}")
else:
    print(f"Process file {data_file}")
print(data_file)

Process file D:\PythonDev\MyQuantFinProject\Data\KeyRiskAsset.csv
D:\PythonDev\MyQuantFinProject\Data\KeyRiskAsset.csv


# Read csv file and extract and transform data as need

In [51]:
df = pd.read_csv(data_file,index_col='Date/Time',parse_dates=['Date/Time'],dayfirst=True)
df.index.set_names('date',inplace=True)
df.sort_index(inplace=True)
df=df.rename(columns={'Ticker':'symbol','close':'price'})
df=df[['symbol','price']]

#test 1
df=df.query("symbol in ('QQQ','GLD','BTC-USD')")
print(df.info())

df.tail(10)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 365 entries, 2023-06-01 to 2023-10-31
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   symbol  365 non-null    object 
 1   price   365 non-null    float64
dtypes: float64(1), object(1)
memory usage: 8.6+ KB
None


,symbol,price
date,,
2023-10-27,BTC-USD,33909.80
2023-10-27,GLD,186.15
2023-10-28,BTC-USD,34089.57
2023-10-29,BTC-USD,34538.48
2023-10-30,GLD,185.10
2023-10-30,QQQ,349.20
2023-10-30,BTC-USD,34502.36
2023-10-31,BTC-USD,34667.78
2023-10-31,GLD,184.09


In [52]:
list_symbol=df['symbol'].unique().tolist() 

print(list_symbol)


['GLD', 'BTC-USD', 'QQQ']


# Basic

# Resample data and calculate Percentage of change

In [57]:
colsROC=['symbol','date','pct_change','std_of_daily_pct','return_to_std']
df_Pct_Price=pd.DataFrame(columns=colsROC)
print(df_Pct_Price)

Empty DataFrame
Columns: [symbol, date, pct_change, std_of_daily_pct, return_to_std]
Index: []


In [58]:
#https://pandas.pydata.org/docs/reference/api/pandas.core.resample.Resampler.aggregate.html
#https://stackoverflow.com/questions/47938372/pandas-dataframe-resample-aggregate-function-use-multiple-columns-with-a-customi
#https://www.alpharithms.com/aggregating-time-series-data-with-pandas-resampling-411212/
# https://sparkbyexamples.com/pandas/pandas-aggregate-functions-with-examples/
for symbol_name in list_symbol:
    print(f"===================================={symbol_name}====================================")
    dfx_symbol=df.query('symbol==@symbol_name')
    dfx_symbol['_pct_compare_prev_']=dfx_symbol['price'].pct_change()
    dfx_symbol=dfx_symbol.loc[from_str:to_str]
    #print(dfx_symbol.tail())
    
    
    dfx_px=dfx_symbol.resample(timeframe).agg({'price': 'last','_pct_compare_prev_':'std'}) 
    dfx_px=dfx_px.rename(columns={'_pct_compare_prev_':'std_of_daily_pct'})

    dfx_px['pct_change']=dfx_px['price'].pct_change()
    dfx_px['pct_change']=dfx_px['pct_change'].round(4)

    dfx_px=dfx_px.dropna() # the first  rown 
    dfx_px=dfx_px.drop(columns=['price']) # remove price becase we are focuse ROC of Price
    
    dfx_px['pct_change']=dfx_px['pct_change']*100
    dfx_px['std_of_daily_pct']=dfx_px['std_of_daily_pct']*100
    
    dfx_px['return_to_std']=dfx_px['pct_change']/dfx_px['std_of_daily_pct']
    
    dfx_px=dfx_px.reset_index()
    dfx_px['symbol']=symbol_name
    dfx_px=dfx_px[colsROC]
    
#     print(f"Calculate ROC of {symbol_name}")
    print(dfx_px)
    
    df_Pct_Price=pd.concat([df_Pct_Price,dfx_px],ignore_index=True,axis=0)

====================================GLD====================================
  symbol       date  pct_change  std_of_daily_pct  return_to_std
0    GLD 2023-08-31       -1.28          0.469818      -2.724459
1    GLD 2023-09-30       -4.76          0.524789      -9.070317
2    GLD 2023-10-31        7.37          0.968912       7.606467
====================================BTC-USD====================================
    symbol       date  pct_change  std_of_daily_pct  return_to_std
0  BTC-USD 2023-08-31      -11.29          2.127747      -5.306082
1  BTC-USD 2023-09-30        4.00          1.171055       3.415723
2  BTC-USD 2023-10-31       28.55          2.368086      12.056151
====================================QQQ====================================
  symbol       date  pct_change  std_of_daily_pct  return_to_std
0    QQQ 2023-08-31       -1.48          1.145144      -1.292413
1    QQQ 2023-09-30       -5.22          0.889364      -5.869360
2    QQQ 2023-10-31       -2.07          1.18

C:\Users\pongthsa\AppData\Local\Temp\ipykernel_20200\1989384564.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx_symbol['_pct_compare_prev_']=dfx_symbol['price'].pct_change()
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_20200\1989384564.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx_symbol['_pct_compare_prev_']=dfx_symbol['price'].pct_change()
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_20200\1989384564.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [59]:
print("==============================================================")
print("List calculate Percentage of change")
print(df_Pct_Price)

List calculate Percentage of change
    symbol       date  pct_change  std_of_daily_pct  return_to_std
0      GLD 2023-08-31       -1.28          0.469818      -2.724459
1      GLD 2023-09-30       -4.76          0.524789      -9.070317
2      GLD 2023-10-31        7.37          0.968912       7.606467
3  BTC-USD 2023-08-31      -11.29          2.127747      -5.306082
4  BTC-USD 2023-09-30        4.00          1.171055       3.415723
5  BTC-USD 2023-10-31       28.55          2.368086      12.056151
6      QQQ 2023-08-31       -1.48          1.145144      -1.292413
7      QQQ 2023-09-30       -5.22          0.889364      -5.869360
8      QQQ 2023-10-31       -2.07          1.189801      -1.739787


# Sum percentage of fund return

In [70]:
print("==============================================================")
print("Sum percentage of fund return")
dfSum_Pct_Price=df_Pct_Price.groupby(['symbol'])[['pct_change']].sum()
dfSum_Pct_Price.reset_index(drop=False,inplace=True)
dfSum_Pct_Price=dfSum_Pct_Price.sort_values(by="pct_change",ascending=False)
dfSum_Pct_Price["pct_change"]=dfSum_Pct_Price["pct_change"].round(1)
print(dfSum_Pct_Price)

Sum percentage of fund return
Empty DataFrame
Columns: [symbol, pct_change]
Index: []


In [71]:
# writer=pd.ExcelWriter( "BasicFundPerformanceReport.xlsx",engine='openpyxl') 

# df_Pct_Price.to_csv(f"ETF-{timeframe}-Return.csv",index=False)
# dfSum_Pct_Price.to_csv(f"ETF-{timeframe}-TotalReturn.csv",index=False)


# Advance

# Pivot Percent Return To Dataframe

In [64]:
print("==============================================================")
print("Pivot Percent Return To Dataframe")
dfPct=df_Pct_Price.pivot(index='date', columns='symbol', values='pct_change')
# print(dfPct.info())
print(dfPct)

symbol,ASP-SME,K-MIDSMALL,KKP-SM-CAP,KKP-TQG,KTMSEQ,TISCOMS-A
date,,,,,,
2023-09-08,-0.93,-0.11,-0.53,-1.04,0.05,0.54
2023-09-15,-0.94,-1.21,-0.53,0.12,-0.63,-0.83
2023-09-22,-0.69,-1.16,-1.07,0.00,-0.80,-0.98
2023-09-29,-2.02,-3.59,-2.29,-3.14,-2.84,-2.53


# Take Percent Return to Find Rank

In [65]:
print("==============================================================")
print("Take Percent Return to Find Rank")
dfRank=dfPct[dfPct.columns.tolist()].rank(method='max',axis=1)
print(dfRank)

symbol,ASP-SME,K-MIDSMALL,KKP-SM-CAP,KKP-TQG,KTMSEQ,TISCOMS-A
date,,,,,,
2023-09-08,2.0,4.0,3.0,1.0,5.0,6.0
2023-09-15,2.0,1.0,5.0,6.0,4.0,3.0
2023-09-22,5.0,1.0,2.0,6.0,4.0,3.0
2023-09-29,6.0,1.0,5.0,2.0,3.0,4.0


# Combine Both return and Rank

In [67]:
print("==============================================================")
print("Combine Both return and Rank")
dfSummary=pd.concat([\
                     dfPct.sum().to_frame(name='sum_return'),\
                     dfRank.sum().to_frame(name='sum_rank'),\
                    ],axis=1)
dfSummary.fillna(value=0,inplace=True)
print(dfSummary)

,sum_return,sum_rank
symbol,,
ASP-SME,-4.58,15.0
K-MIDSMALL,-6.07,7.0
KKP-SM-CAP,-4.42,15.0
KKP-TQG,-4.06,15.0
KTMSEQ,-4.22,16.0
TISCOMS-A,-3.80,16.0


# Summarize Total Score 

In [68]:
print("==============================================================")
print("Summarize Total Score ")
dfFinalScore=dfSummary.rank(method='max',axis=0)
dfFinalScore['TotalScore']=dfFinalScore.sum(axis=1)
dfFinalScore.sort_values(by='TotalScore',ascending=False,inplace=True)
print(dfFinalScore)

,sum_return,sum_rank,TotalScore
symbol,,,
TISCOMS-A,6.0,6.0,12.0
KTMSEQ,4.0,6.0,10.0
KKP-TQG,5.0,4.0,9.0
KKP-SM-CAP,3.0,4.0,7.0
ASP-SME,2.0,4.0,6.0
K-MIDSMALL,1.0,1.0,2.0


In [69]:
if isPriceExcel:
    print("Export outcoome as Excel")
    with pd.ExcelWriter(f"{report_prefix_name}_{timeframe}_{from_str}To{to_str}.xlsx") as writer:
        dfPct.to_excel(writer, sheet_name=f"{timeframe}-Return")
        dfSum_Pct_Price.to_excel(writer, sheet_name="Return")
        dfRank.to_excel(writer, sheet_name="RANK")
        dfSummary.to_excel(writer, sheet_name="Summary")
        dfFinalScore.to_excel(writer, sheet_name="FinalScore")